<a href="https://colab.research.google.com/github/Kristin33/Composer-Clustering/blob/master/auto_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# get the dataset from git
!git clone https://github.com/Kristin33/Composer-Clustering

%cd Composer-Clustering

fatal: destination path 'Composer-Clustering' already exists and is not an empty directory.
/content/Composer-Clustering


In [2]:
!pip install pretty_midi

import os, sys
import pretty_midi
import librosa, librosa.display
import matplotlib.pyplot as plt 

from scipy import stats

import numpy as np 
import pandas as pd 

     |████████████████████████████████| 5.6MB 2.3MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.8-cp36-none-any.whl size=5590819 sha256=267a3839f51fd14755b67b39ce018fe6c14478fd93508fa4e9e7ccb86c8d946f
  Stored in directory: /root/.cache/pip/wheels/22/e7/6b/70eb5879f7dbcb4f44fee735a61d6298f9e082be8538b52422
Successfully built pretty-midi


# Data directories and experiment

In [0]:
# directory variables 

alkan_dir = "alkan_(c)contributors-kunstderfuge/"
bach_dir = "bach_lute_(c)contributors-kunstderfuge/"
dandrieu_dir = "dandrieu_(c)contributors-kunstderfuge/"
dvorak_dir = "dvorak_(c)contributors-kunstderfuge/"
scriabin_dir = "scriabin_(c)contributors-kunstderfuge/"
byrd_dir = "byrd_(c)contributors-kunstderfuge/"
faure_dir = "faure_(c)contributors-kunstderfuge/"
buxtehude_dir = "buxtehude_(c)contributors-kunstderfuge/"
beethoven_dir = "beethoven_iii_(c)contributors-kunstderfuge/"
schumann_dir = "schumann_(c)contributors-kunstderfuge/"
scriabin_dir = "scriabin_(c)contributors-kunstderfuge/"
shostakovich_dir = "shostakovich_(c)contributors-kunstderfuge/"
soler_dir = "soler_(c)contributors-kunstderfuge/"

train_comps = [bach_dir, beethoven_dir]
test_comps = [scriabin_dir, buxtehude_dir, byrd_dir]


alkan_csv = "alkan_all.csv"
bach_csv = "bach_all.csv"
beethoven_csv = "beethoven_all.csv"
brahms_csv = "brahms_all.csv"
buxtehude_csv = "buxtehude_all.csv"
byrd_csv = "byrd_all.csv"
chopin_csv = "chopin_all.csv"
dandrieu_csv = "dandrieu_all.csv"
dvorak_csv = "dvorak_all.csv"
faure_csv = "faure_all.csv"
handel_csv = "handel_all.csv"
haydn_csv = "haydn_all.csv"
mozart_csv = "mozart_all.csv"
scarlatti_csv = "scarlatti_all.csv"
schubert_csv = "schubert_all.csv"
schumann_csv = "schumann_all.csv"
scriabin_csv = "scriabin_all.csv"
shostakovich_csv = "shostakovich_all.csv"
soler_csv = "soler_all.csv"

train_csv_comps = [alkan_csv, bach_csv, beethoven_csv,
                   brahms_csv, buxtehude_csv, byrd_csv, 
                   chopin_csv, dandrieu_csv, dvorak_csv,
                   faure_csv, handel_csv, haydn_csv,
                   mozart_csv, scarlatti_csv, schubert_csv,
                  schumann_csv, scriabin_csv, shostakovich_csv, soler_csv]
test_csv_comps = [alkan_csv, bach_csv, beethoven_csv,
                   brahms_csv]
                  # , buxtehude_csv, byrd_csv, 
                  #  chopin_csv, dandrieu_csv, dvorak_csv,
                  #  faure_csv, handel_csv, haydn_csv,
                  #  mozart_csv, scarlatti_csv, schubert_csv,
                  # schumann_csv, scriabin_csv, shostakovich_csv, soler_csv]


In [4]:
# test_comps = [scriabin_dir, buxtehude_dir, byrd_dir]
f1_e1 = [dandrieu_dir, soler_dir]
f1_e2 = [dvorak_dir, schumann_dir]
f1_e3 = [buxtehude_dir, faure_dir]
f1_e4 = [scriabin_dir, byrd_dir]
# f1_e5 = [shostakovich_dir, brahms_dir]
# f1_e6 = [chopin_dir, debussy_dir]
# f1_e7 = [schubert_dir, alkan_dir]
# f1_e8 = [handel_dir, mozart_dir]
# f1_e9 = [haydn_dir, beethoven_dir]
# f1_e10 = [scarlatti_dir, bach_dir]

NameError: ignored

# Fetch Data Functions

In [0]:
"""
===========================================================
Get the piano roll encoding of midi files 
===========================================================

Attempt of using autoencoders to encode the pianoroll representation
into a lower dimension latent representation. 

currently, the input of the piano roll is of dimension 1280000

"""

def get_piano_roll_matrix(midi_data, start_pitch, end_pitch, fs=50, draw=False):
    # roll = midi_data.get_piano_roll(fs)[start_pitch:end_pitch]
    matrix = midi_data.get_piano_roll(fs)[:, :10000]
    # print(matrix[:, 30:40])
    # print(matrix.shape)

    if draw: 
      librosa.display.specshow(matrix,
            hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
            fmin=pretty_midi.note_number_to_hz(start_pitch))
    return np.array(matrix).flatten()

# input: an array of 
def get_test_data(test_comps):
    
    data_dir = "New_Data_Selection/"

    comp_data, comp_label = [], []

    for idx, test_comp in enumerate(test_comps):
      print(test_comp)
      tmp_data, tmp_label = [], []
      for filename in os.listdir(data_dir + test_comp):
          
          if ".mid" in filename or ".MID" in filename:
              midi_data = pretty_midi.PrettyMIDI(data_dir + test_comp + filename)
              l = midi_data.get_end_time()
              # scale the sampling frequency by the length of data, so the picture is 
              # of the same size 128 * 10000
              fs = 50 * (10000/(l * 50 - 1))
              roll = []
              roll.append(get_piano_roll_matrix(midi_data,36,108,fs=fs,draw=False))
              tmp = tmp_data + roll
              if (len(np.array(tmp).shape) != 2):
                continue
              tmp_data.append(roll[0])
              tmp_label.append(idx)
      comp_data.extend(tmp_data)
      comp_label.extend(tmp_label)


    data = np.array(comp_data)
    labels = np.array(comp_label)

    return data, labels


def get_train_data(train_comps):
    
    data_dir = "New_Data_Selection/"

    comp_data, comp_label = [], []

    for idx, train_comp in enumerate(train_comps):
      print(train_comp)
      tmp_data, tmp_label = [], []
      for filename in os.listdir(data_dir + train_comp):
          if ".mid" in filename or ".MID" in filename:
              midi_data = pretty_midi.PrettyMIDI(data_dir + train_comp + filename)
              l = midi_data.get_end_time()
              # scale the sampling frequency by the length of data, so the picture is 
              # of the same size 128 * 10000
              fs = 50 * (10000/(l * 50 - 1))
              roll = []
              roll.append(get_piano_roll_matrix(midi_data,36,108,fs=fs,draw=False))
              tmp = tmp_data + roll
              if (len(np.array(tmp).shape) != 2):
                continue
              tmp_data.append(roll[0])
              tmp_label.append(idx)
      comp_data.extend(tmp_data)
      comp_label.extend(tmp_label)

    data = np.array(comp_data)
    labels = np.array(comp_label)

    return data, labels



# Fetch data: old csv files

In [0]:
def get_csv_train_data(train_csv_comps):
    
    data_dir = "0229_Experiment/csv/"

    composer_data = np.empty((0, 776))
    composer_label = np.empty((0,))

    for filename in train_csv_comps:
        data = np.genfromtxt(data_dir + filename, delimiter=',', 
                             usecols=np.arange(1,777), encoding="latin1")
        # get rid of the top row with feature names
        data = data[1:,:]
        # standarize the data
        data = stats.zscore(data)
        data = np.nan_to_num(data)

        composer_data = np.append(composer_data, data, axis=0)
        composer_label = np.append(composer_label, np.zeros((data.shape[0],)), axis=0)

    data = composer_data
    labels = composer_label

    return data, labels

def get_csv_test_data(test_csv_comps):
    
    data_dir = "0229_Experiment/csv/"

    composer_data = np.empty((0, 776))
    composer_label = np.empty((0,))
    

    for idx, filename in enumerate(test_csv_comps):
        print(filename)
        data = np.genfromtxt(data_dir + filename, delimiter=',', 
                             usecols=np.arange(1,777), encoding='latin1')
        # get rid of the top row with feature names
        data = data[1:,:]
        # standarize the data
        data = stats.zscore(data)
        data = np.nan_to_num(data)

        composer_data = np.append(composer_data, data, axis=0)
        composer_label = np.append(composer_label, np.ones((data.shape[0],)) * idx, axis=0)

    data = composer_data
    labels = composer_label

    return data, labels


# Fetch training data for autoencoders

In [0]:
"""
===========================================================
Auto encoders
===========================================================

Attempt of using autoencoders to encode the pianoroll representation
into a lower dimension latent representation. 

currently, the input of the piano roll is of dimension 1280000


"""

import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

%tensorflow_version 1.x
import tensorflow as tf
# from __future__ import print_function
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam
from keras import regularizers




In [0]:
(x_train, y_train) = get_train_data()

x_train = x_train.astype('float32') / 255.0

x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))


print(x_train.shape)

# Construct Model and Train

In [0]:

def get_ae(input_size):
  # if using piano roll, input_size = 1280000, if jSymbolic feature, size is 776 
  hidden_size = 128
  code_size = 32

  input_img = Input(shape=(input_size,))
  hidden_1 = Dense(hidden_size, activation='relu')(input_img)
  code = Dense(code_size, activation='relu')(hidden_1)
  hidden_2 = Dense(hidden_size, activation='relu')(code)
  output_img = Dense(input_size, activation='sigmoid')(hidden_2)


  # input_img = Input(shape=(input_size,))
  # encoded = Dense(128, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_img)
  # encoded = Dense(64, activation='relu', activity_regularizer=regularizers.l1(10e-5))(encoded)
  # encoded = Dense(32, activation='relu', activity_regularizer=regularizers.l1(10e-5))(encoded)
  # code = Dense(8, activation='relu', activity_regularizer=regularizers.l1(10e-5))(encoded)
  # decoded = Dense(32, activation='relu')(code)
  # decoded = Dense(64, activation='relu')(decoded)
  # decoded = Dense(128, activation='relu')(decoded)
  # output_img = Dense(input_size, activation='sigmoid')(decoded)


  autoencoder = Model(input_img, output_img)
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  autoencoder_half = Model(input_img, code)

  return autoencoder, autoencoder_half



In [0]:
# alternative model: Convolutional Autoencoders
from keras.layers import Input, Dense, Reshape, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

def get_conv_ae(input_size):
  # if using piano roll, input_size = 1280000, if jSymbolic feature, size is 776 

  input_img = Input(shape=(120, 9992, 1))

  x = Conv2D(16, (5, 5), activation='relu', padding='same')(input_img)
  x = MaxPooling2D((2, 2), padding='same')(x)
  x = Conv2D(8, (5, 5), activation='relu', padding='same')(x)
  x = MaxPooling2D((2, 2), padding='same')(x)
  x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
  x = MaxPooling2D((2, 2), padding='same')(x)
  x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
  encoded = MaxPooling2D((2, 2), padding='same')(x)

  # at this point the representation is (8, 625, 8) i.e. 400000-dimensional
  tmp = Reshape((40000,), input_shape=(8, 625, 8))(encoded)
  hidden_1 = Dense(128, activation='relu')(tmp)
  code = Dense(32, activation='relu')(hidden_1)
  # code should have dimension 32? 
  hidden_2 = Dense(128, activation='relu')(code)
  hidden_3 = Dense(40000, activation='relu')(hidden_2)
  tmp = Reshape((8, 625, 8), input_shape=(40000,))(hidden_3)

  x = Conv2D(8, (3, 3), activation='relu', padding='same')(tmp)
  x = UpSampling2D((2, 2))(x)
  x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
  x = UpSampling2D((2, 2))(x)
  x = Conv2D(8, (5, 5), activation='relu', padding='same')(x)
  x = UpSampling2D((2, 2))(x)
  x = Conv2D(16, (5, 5), activation='relu')(x)
  x = UpSampling2D((2, 2))(x)
  decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

  autoencoder = Model(input_img, decoded)
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

  return autoencoder



In [0]:
# train autoencoder 


# if using piano roll, input_size = 1280000, if jSymbolic feature, size is 776 
input_size = 1280000
# autoencoder, autoencoder_half = get_ae(input_size)
# autoencoder_train = autoencoder.fit(x_train, x_train, epochs=10)

autoencoder = get_conv_ae(input_size)
# x_train = np.reshape(x_train, (len(x_train), 1, 128, 10000))
a = np.reshape(x_train, (len(x_train), 128, 10000, 1))
x_train_new = a[:, 8:, 8:, :]
autoencoder_train = autoencoder.fit(x_train_new, x_train_new,
                nb_epoch=100)





# Parameter Search (tbd)

The parameters that we want to tune: epoches, hidden layer size, and code size. 

Model variation that we want to try: sparse autoendoers, deep autoencoders, convolutional autoencoders 

In [0]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline

# auto = Pipeline([
#     ('auto',autoencoder)
# ])
# auto.get_params()

In [0]:


# # batch_size = [10, 20, 40, 60, 80, 100]
# hidden_size = [64, 128, 256]
# code_size = [32, 64, 128]
# epochs = [4, 10, 50, 100]
# # param_grid = dict(batch_size=batch_size, epoch=epochs)
# param_grid = dict(hidden_size=hidden_size, code_size=code_size, epoch=epochs)
# grid = GridSearchCV(estimator=auto, param_grid=param_grid, n_jobs=1, scoring="accuracy")
# grid_result = grid.fit(x_train, x_train)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


# Fetch testing data and cluster

In [0]:

def get_latent_representation(expr):

  test_comps = expr
  # test_comps = [scriabin_dir, buxtehude_dir, byrd_dir]

  (x_test, y_test) = get_test_data(test_comps)
  # if using piano roll, x_test.shape should be n * 1280000, n is the number of 
  # test samples. If using csv, x_test.shape should be n * 776
  print(x_test.shape)
  x_test = x_test.astype('float32') / 255.0
  x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

  # using autoencoders

  # using convolutional autoencoder, needs to reshape data (input and output)
  autoencoder_half = Model(inputs=autoencoder.input, outputs=autoencoder.layers[11].output)
  a = np.reshape(x_test, (len(x_test), 128, 10000, 1))
  x_test_new = a[:, 8:, 8:, :]
  pred = autoencoder_half.predict(x_test_new)
  print(pred.shape)

  # just using the matrix data to cluster
  # pred = x_test
  # print(pred.shape)

  data = pred
  labels = y_test

  np.random.seed(42)

  # X_digits, y_digits = load_digits(return_X_y=True)
  # print("X_digits shape: {}".format(X_digits.shape))
  # print("y_digits shape: {}".format(y_digits.shape))
  # data = scale(X_digits)
  print("data shape: {}".format(data.shape))

  n_samples, n_features = data.shape
  n_digits = len(np.unique(labels))
  # n_digits = len(np.unique(y_digits))
  # labels = y_digits


  print("n_composers: %d, \t n_test_samples %d, \t n_features %d"
        % (n_digits, n_samples, n_features))
  
  return data

In [0]:
data = pred
labels = y_test

np.random.seed(42)

# X_digits, y_digits = load_digits(return_X_y=True)
# print("X_digits shape: {}".format(X_digits.shape))
# print("y_digits shape: {}".format(y_digits.shape))
# data = scale(X_digits)
print("data shape: {}".format(data.shape))

n_samples, n_features = data.shape
n_digits = len(np.unique(labels))
# n_digits = len(np.unique(y_digits))
# labels = y_digits


print("n_composers: %d, \t n_test_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))


In [0]:
from time import time

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale



# Run KMeans

In [0]:
# experiment is a list of composer directories that we are 
# fetching test data from
def bench_k_means(estimator, name, data, experiment):
    data = get_latent_representation(experiment)

    t0 = time()
    estimator.fit(data)
    print(labels)
    print(estimator.labels_)
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_)))
            #  metrics.silhouette_score(data, estimator.labels_,
            #                           metric='euclidean')))
    expr_name = "_".join(experiment)
    with open(expr_name + ".txt", "w") as f:
      f.write("experiment: " + expr_name + "\n")
      f.write("algorithm: convolutional autoencoders \n")
      f.write("training data: bach lute and beethoven piano concertos(26) \n")
      f.write("latent representation: dimension 32 vector \n")
      f.write('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette\n')
      f.write('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_)))
      f.write(82 * "_" + "\n")
      f.write(np.array2string(labels))
      f.write("\n")
      f.write(np.array2string(estimator.labels_))

expr = f1e1







In [0]:


expr_list = [f1_e1, f1_e2, f1_e3, f1_e4]
            #  f1_e5, f1_e6, f1_e7, f1_e8, f1_e9, f1_e10]

# expr_list is a list of experiments, experiment is a list of composition directories
def run_batch_experiments(expr_list):
  for expr in expr_list:

    print("\n {} \n".format(*expr))
    print(82 * '_')
    print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
                  name="k-means++", data=data, experiment=expr)

    bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
                  name="random", data=data, experiment=expr)

    # in this case the seeding of the centers is deterministic, hence we run the
    # kmeans algorithm only once with n_init=1
    pca = PCA(n_components=n_digits).fit(data)
    bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
                  name="PCA-based",
                  data=data, experiment=expr)
    print(82 * '_')
    

In [0]:
!ls

# Visualization 

In [0]:
# #############################################################################
# Visualize the results on PCA-reduced data

reduced_data = PCA(n_components=2).fit_transform(data)
kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=10)
kmeans.fit(reduced_data)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, x_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

# draw the data points 
# TODO: currently these data points are all black, want to color them by 
# the true composers
plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)


# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the latent representation of compositions (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()